In [1]:
from dotenv import load_dotenv, find_dotenv 

from langchain_openai import ChatOpenAI
from langchain.schema import SystemMessage
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate

In [2]:
load_dotenv(find_dotenv(), override=True)

True

In [4]:
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1)

prompt = ChatPromptTemplate(
    input_variables=["content"],
    messages=[
        SystemMessage(content='You respond only in Portuguese.'),
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True
)

In [9]:
while True:
    content = input('Your prompt: ')

    if content.lower() in ['quit', 'exit', 'bye']:
        print('Goodbye!')
        break
    
    response = chain.invoke({'content': content})

    print(response)
    print('=' * 100)



> Entering new LLMChain chain...
Prompt after formatting:
System: You respond only in Portuguese.
Human: Olá, como vai?

> Finished chain.
{'content': 'Olá, como vai?', 'text': 'Olá! Estou bem, obrigado. E você, como está? Posso te ajudar com alguma coisa?'}


> Entering new LLMChain chain...
Prompt after formatting:
System: You respond only in Portuguese.
Human: Gostaria de saber mais sobre Blockchain

> Finished chain.
{'content': 'Gostaria de saber mais sobre Blockchain', 'text': 'Claro! O blockchain é uma tecnologia de registro distribuído que permite armazenar e compartilhar informações de forma segura e transparente. Ele é frequentemente associado às criptomoedas, como o Bitcoin, pois é a tecnologia por trás da sua operação. No entanto, o blockchain tem aplicações em diversos outros setores, como logística, saúde, votação eletrônica, entre outros. Ele funciona através de blocos de informações encadeados de forma criptografada, garantindo a integridade e a imutabilidade dos dado

## Adding Memory

In [10]:
from langchain.memory import ConversationBufferMemory
from langchain.prompts import MessagesPlaceholder


llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)


memory = ConversationBufferMemory(
    memory_key='chat_history',
    return_messages=True
)

prompt = ChatPromptTemplate(
    input_variables=["content", "chat_history"],
    messages=[
        SystemMessage(content="You are a chatbot having a conversation with a human."),
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=False
)

In [11]:
while True:
    content = input('Your prompt: ')

    if content.lower() in ['quit', 'exit', 'bye']:
        print('Goodbye!')
        break
    
    response = chain.invoke({'content': content})

    print(response)
    print('=' * 100)

{'content': 'Olá, qual seu nome?', 'chat_history': [HumanMessage(content='Olá, qual seu nome?'), AIMessage(content='Olá! Eu sou um assistente virtual, não tenho um nome específico. Como posso ajudar você hoje?')], 'text': 'Olá! Eu sou um assistente virtual, não tenho um nome específico. Como posso ajudar você hoje?'}
{'content': 'Im 50 years old', 'chat_history': [HumanMessage(content='Olá, qual seu nome?'), AIMessage(content='Olá! Eu sou um assistente virtual, não tenho um nome específico. Como posso ajudar você hoje?'), HumanMessage(content='Im 50 years old'), AIMessage(content="That's great! How are you feeling about being 50?")], 'text': "That's great! How are you feeling about being 50?"}
{'content': 'How old am I?', 'chat_history': [HumanMessage(content='Olá, qual seu nome?'), AIMessage(content='Olá! Eu sou um assistente virtual, não tenho um nome específico. Como posso ajudar você hoje?'), HumanMessage(content='Im 50 years old'), AIMessage(content="That's great! How are you feel

## Saving Chat Sessions

In [12]:
from langchain.memory import FileChatMessageHistory

llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1)

history = FileChatMessageHistory('chat_history.json')

memory = ConversationBufferMemory(
    memory_key='chat_history',
    chat_memory=history,
    return_messages=True
)

prompt = ChatPromptTemplate(
    input_variables=["content", "chat_history"],
    messages=[
        SystemMessage(content="You are a chatbot having a conversation with a human."),
        MessagesPlaceholder(variable_name="chat_history"), 
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory,
    verbose=False
)

In [13]:
while True:
    content = input('Your prompt: ')
    if content.lower() in ['quit', 'exit', 'bye']:
        print('Goodbye!')
        break
    
    response = chain.invoke({'content': content})

    print(response)
    print('=' * 100)

{'content': 'Olá, como está?', 'chat_history': [], 'text': 'Olá! Estou aqui e pronto para conversar. Como posso ajudá-lo hoje?'}
{'content': 'Me ajude a iniciar um código FastAPI em python?', 'chat_history': [HumanMessage(content='Olá, como está?'), AIMessage(content='Olá! Estou aqui e pronto para conversar. Como posso ajudá-lo hoje?')], 'text': 'Claro! Vou te mostrar um exemplo simples de código FastAPI em Python para te ajudar a começar. Aqui está um exemplo de um arquivo chamado `main.py`:\n\n```python\nfrom fastapi import FastAPI\n\n# Criar uma instância do FastAPI\napp = FastAPI()\n\n# Rota padrão\n@app.get("/")\nasync def read_root():\n    return {"Hello": "World"}\n\n# Rota com parâmetro\n@app.get("/items/{item_id}")\nasync def read_item(item_id: int, q: str = None):\n    return {"item_id": item_id, "q": q}\n\n# Executar o servidor\nif __name__ == "__main__":\n    import uvicorn\n    uvicorn.run(app, host="127.0.0.1", port=8000)\n```\n\nPara executar o código, você precisará ins

In [14]:
print(history.messages)

[HumanMessage(content='Olá, como está?'), AIMessage(content='Olá! Estou aqui e pronto para conversar. Como posso ajudá-lo hoje?'), HumanMessage(content='Me ajude a iniciar um código FastAPI em python?'), AIMessage(content='Claro! Vou te mostrar um exemplo simples de código FastAPI em Python para te ajudar a começar. Aqui está um exemplo de um arquivo chamado `main.py`:\n\n```python\nfrom fastapi import FastAPI\n\n# Criar uma instância do FastAPI\napp = FastAPI()\n\n# Rota padrão\n@app.get("/")\nasync def read_root():\n    return {"Hello": "World"}\n\n# Rota com parâmetro\n@app.get("/items/{item_id}")\nasync def read_item(item_id: int, q: str = None):\n    return {"item_id": item_id, "q": q}\n\n# Executar o servidor\nif __name__ == "__main__":\n    import uvicorn\n    uvicorn.run(app, host="127.0.0.1", port=8000)\n```\n\nPara executar o código, você precisará instalar o FastAPI e o Uvicorn. Você pode fazer isso executando os seguintes comandos no seu terminal:\n\n```\npip install fastap

In [15]:
import pandas as pd

In [16]:
pd.read_json('chat_history.json')

,type,data
0,human,"{'content': 'Olá, como está?', 'additional_kwa..."
1,ai,{'content': 'Olá! Estou aqui e pronto para con...
2,human,{'content': 'Me ajude a iniciar um código Fast...
3,ai,{'content': 'Claro! Vou te mostrar um exemplo ...
4,human,"{'content': 'Como enviar dados?', 'additional_..."
5,ai,{'content': 'Para enviar dados para o servidor...
